# Overview
- nb004 の拡張版
- n_feat/m_audio/1_bird

# Const

In [1]:
NB = '007'
CREATE_FEATSETS_NAME = 'librosa_mfcc02'
DIR_FEAT = './../data_ignore/features/table'
DIR_AUDIO = './../data_ignore/official/train_audio'
PATH_TRAIN_CSV = './../data_ignore/official/train.csv'

# Import everything I need :)

In [2]:
import warnings
warnings.filterwarnings('ignore')
import glob
import pandas as pd
import numpy as np
import librosa
from fastprogress import progress_bar
import matplotlib.pyplot as plt
import seaborn as sns

# My function

In [3]:
def extract_mfccs(x, sr, n_fft, hop_length, n_mels):
    mfccs = librosa.feature.mfcc(y=x,
                                 sr=sr,
                                 n_fft=n_fft,
                                 hop_length=hop_length,
                                 center=False,
                                 n_mels=n_mels)
    feat_name = [f'librosa_mfcc_{i+1}' for i in range(n_mels)]
    return mfccs.T, feat_name

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in progress_bar(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Preparation

settings

In [4]:
pd.set_option('display.max_columns', 100)

<br>
load train.csv

In [5]:
df_train = pd.read_csv(PATH_TRAIN_CSV)
df_train.head(2)

,rating,playback_used,ebird_code,channels,date,pitch,duration,filename,speed,species,number_of_notes,title,secondary_labels,bird_seen,sci_name,location,latitude,sampling_rate,type,elevation,description,bitrate_of_mp3,file_type,volume,background,xc_id,url,country,author,primary_label,longitude,length,time,recordist,license
0,3.5,no,aldfly,1 (mono),2013-05-25,Not specified,25,XC134874.mp3,Not specified,Alder Flycatcher,Not specified,XC134874 Alder Flycatcher (Empidonax alnorum),"['Empidonax minimus_Least Flycatcher', 'Leioth...",yes,Empidonax alnorum,"Grey Cloud Dunes SNA, Washington, Minnesota",44.793,48000 (Hz),call,220 m,Two short segments of fairly quiet *pip* calls...,192000 (bps),mp3,Not specified,American Yellow Warbler (Setophaga aestiva); L...,134874,https://www.xeno-canto.org/134874,United States,Jonathon Jongsma,Empidonax alnorum_Alder Flycatcher,-92.962,Not specified,8:00,Jonathon Jongsma,Creative Commons Attribution-ShareAlike 3.0
1,4.0,no,aldfly,2 (stereo),2013-05-27,both,36,XC135454.mp3,both,Alder Flycatcher,1-3,XC135454 Alder Flycatcher (Empidonax alnorum),[],yes,Empidonax alnorum,"Carver's Gap Parking area, Roan Mountain Stat...",36.1065,44100 (Hz),"call, song",1700 m,At least three birds seen here moving around s...,128000 (bps),mp3,level,NaN,135454,https://www.xeno-canto.org/135454,United States,Mike Nelson,Empidonax alnorum_Alder Flycatcher,-82.1106,0-3(s),08:30,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...


# Create features

In [6]:
dir_birds = glob.glob(f'{DIR_AUDIO}/*')
dir_birds = sorted(dir_birds)
dir_birds[:3]

['./../data_ignore/official/train_audio/aldfly',
 './../data_ignore/official/train_audio/ameavo',
 './../data_ignore/official/train_audio/amebit']

- librosa の mfcc_1 を抽出
- 1つのwavで平均した mfcc_* を出す
- m_feat/m_audio/1_bird だけ特徴量抽出してみる

In [ ]:
df_list = []
for dir_bird in progress_bar(dir_birds):
    audio_files = glob.glob(f'{dir_bird}/*')
    audio_files = sorted(audio_files)
#     for audio_file in audio_files[:1]:
    for audio_file in audio_files:
        window_size = 0.5  # sec
        signal, sr = librosa.load(audio_file, sr=None)
        
        n_fft = int(sr * window_size)
        if n_fft > len(signal):
            n_fft = len(signal)
        hop_length = int(n_fft/2)
        feat_val, feat_name = extract_mfccs(signal,
                                            sr=sr,
                                            n_fft=n_fft,
                                            hop_length=hop_length,
                                            n_mels=12,
                                           )
#         feat_val = feat_val.mean(axis=0)
        df_feat = pd.DataFrame(feat_val, columns=feat_name)
        df_feat = df_feat.drop(['librosa_mfcc_1'], axis=1)
        df_feat['birds'] = dir_bird.split('/')[-1]
        df_feat['filename'] = audio_file.split('/')[-1]
        df_list.append(df_feat)
        
df_merge01 = pd.concat(df_list)
df_merge01 = df_merge01.reset_index(drop=True)

In [ ]:
df_merge01 = reduce_mem_usage(df_merge01)

In [ ]:
df_merge01['filename'] = df_train['filename']
df_merge01.tail()

# Save

In [ ]:
filename = f'nb{NB}_{CREATE_FEATSETS_NAME}.csv'
save_path = f'{DIR_FEAT}/{filename}'
save_path

In [ ]:
df_merge01.to_csv(save_path, index=False, encoding='utf-8')